In [1]:
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

#Feature tools for "Deep Feature Synthesis"
import featuretools as ft

import pyodbc

## Deep Feature Synthesis
Deep Feature Synthesis (DFS) is an automated method for performing feature engineering on relational and temporal data.

### Input Data
Deep Feature Synthesis requires structured datasets in order to perform feature engineering. To demonstrate the capabilities of DFS, we will use a mock customer transactions dataset.

__Note:__
Before using DFS, it is recommended that you prepare your data as an `EntitySet`. See [Representing Data with EntitySets](https://docs.featuretools.com/loading_data/using_entitysets.html) to learn how.

In [2]:
es = ft.demo.load_mock_customer(return_entityset=True)

In [3]:
es

Entityset: transactions
  Entities:
    transactions [Rows: 500, Columns: 5]
    products [Rows: 5, Columns: 2]
    sessions [Rows: 35, Columns: 4]
    customers [Rows: 5, Columns: 4]
  Relationships:
    transactions.product_id -> products.product_id
    transactions.session_id -> sessions.session_id
    sessions.customer_id -> customers.customer_id

### Running DFS
Typically, without automated feature engineering, a data scientist would write code to aggregate data for a customer, and apply different statistical functions resulting in features quantifying the customer’s behavior. In this example, an expert might be interested in features such as: total number of sessions or month the customer signed up.

These features can be generated by DFS when we specify the target_entity as `customers` and `"count"` and `"month"` as primitives.

In [4]:
feature_matrix, feature_defs = ft.dfs(entityset=es,
                                        target_entity="customers",
                                        agg_primitives=["count"],
                                        trans_primitives=["month"],
                                        max_depth=1)

In [5]:
feature_matrix

,zip_code,COUNT(sessions),MONTH(date_of_birth),MONTH(join_date)
customer_id,,,,
5,60091,6,7,7
4,60091,8,8,4
1,60091,8,7,4
3,13244,6,11,8
2,13244,7,8,4


In the example above, `"count"` is an __aggregation primitive__ because it computes a single value based on many sessions related to one customer. `"month"` is called a __transform primitive__ because it takes one value for a customer transforms it to another.

### Creating “Deep Features”
The name Deep Feature Synthesis comes from the algorithm’s ability to stack primitives to generate more complex features. Each time we stack a primitive we increase the “depth” of a feature. The `max_depth` parameter controls the maximum depth of the features returned by DFS. Let us try running DFS with `max_depth=2`

In [6]:
feature_matrix, feature_defs = ft.dfs(entityset=es,
                                        target_entity="customers",
                                        agg_primitives=["mean", "sum", "mode"],
                                        trans_primitives=["month", "hour"],
                                        max_depth=2)

In [7]:
feature_matrix

,zip_code,MODE(sessions.device),MEAN(transactions.amount),SUM(transactions.amount),MODE(transactions.product_id),MONTH(date_of_birth),MONTH(join_date),HOUR(date_of_birth),HOUR(join_date),MEAN(sessions.MEAN(transactions.amount)),MEAN(sessions.SUM(transactions.amount)),SUM(sessions.MEAN(transactions.amount)),MODE(sessions.MODE(transactions.product_id)),MODE(sessions.HOUR(session_start)),MODE(sessions.MONTH(session_start)),MODE(transactions.sessions.customer_id),MODE(transactions.sessions.device)
customer_id,,,,,,,,,,,,,,,,,
5,60091,mobile,80.375443,6349.66,5,7,7,0,5,78.705187,1058.276667,472.231119,3,0,1,5,mobile
4,60091,mobile,80.070459,8727.68,2,8,4,0,20,81.207189,1090.960000,649.657515,1,1,1,4,mobile
1,60091,mobile,71.631905,9025.62,4,7,4,0,10,72.774140,1128.202500,582.193117,4,6,1,1,mobile
3,13244,desktop,67.060430,6236.62,1,11,8,0,15,67.539577,1039.436667,405.237462,1,5,1,3,desktop
2,13244,desktop,77.422366,7200.28,4,8,4,0,23,78.415122,1028.611429,548.905851,3,3,1,2,desktop


With a depth of 2, a number of features are generated using the supplied primitives. The algorithm to synthesize these definitions is described in this paper. In the returned feature matrix, let us understand one of the depth 2 features

In [8]:
feature_matrix[['MEAN(sessions.SUM(transactions.amount))']]

,MEAN(sessions.SUM(transactions.amount))
customer_id,
5,1058.276667
4,1090.960000
1,1128.202500
3,1039.436667
2,1028.611429


For each customer this feature

1. calculates the sum of all transaction amounts per session to get total amount per session,

2. then applies the mean to the total amounts across multiple sessions to identify the average amount spent per session

We call this feature a “deep feature” with a depth of 2.

Let’s look at another depth 2 feature that calculates for every customer the most common hour of the day when they start a session

In [9]:
feature_matrix[['MODE(sessions.HOUR(session_start))']]

,MODE(sessions.HOUR(session_start))
customer_id,
5,0
4,1
1,6
3,5
2,3


For each customer this feature calculates

1. The hour of the day each of his or her sessions started, then

2. uses the statistical function mode to identify the most common hour he or she started a session

Stacking results in features that are more expressive than individual primitives themselves. This enables the automatic creation of complex patterns for machine learning.

### Changing Target Entity 
DFS is powerful because we can create a feature matrix for any entity in our dataset. If we switch our target entity to “sessions”, we can synthesize features for each session instead of each customer. Now, we can use these features to predict the outcome of a session.

In [11]:
feature_matrix, feature_defs = ft.dfs(entityset=es,
                                        target_entity="sessions",
                                        agg_primitives=["mean", "sum", "mode"],
                                        trans_primitives=["month", "hour"],
                                        max_depth=2)

In [12]:
feature_matrix.head(5)

,customer_id,device,MEAN(transactions.amount),SUM(transactions.amount),MODE(transactions.product_id),MONTH(session_start),HOUR(session_start),customers.zip_code,MODE(transactions.HOUR(transaction_time)),MODE(transactions.products.brand),MODE(transactions.MONTH(transaction_time)),customers.MODE(sessions.device),customers.MEAN(transactions.amount),customers.SUM(transactions.amount),customers.MODE(transactions.product_id),customers.MONTH(date_of_birth),customers.MONTH(join_date),customers.HOUR(date_of_birth),customers.HOUR(join_date)
session_id,,,,,,,,,,,,,,,,,,,
1,2,desktop,76.813125,1229.01,3,1,0,13244,0,B,1,desktop,77.422366,7200.28,4,8,4,0,23
2,5,mobile,74.696000,746.96,5,1,0,60091,0,A,1,mobile,80.375443,6349.66,5,7,7,0,5
3,4,mobile,88.600000,1329.00,1,1,0,60091,0,B,1,mobile,80.070459,8727.68,2,8,4,0,20
4,1,mobile,64.557200,1613.93,5,1,0,60091,0,B,1,mobile,71.631905,9025.62,4,7,4,0,10
5,4,mobile,70.638182,777.02,5,1,1,60091,1,B,1,mobile,80.070459,8727.68,2,8,4,0,20


As we can see, DFS will also build deep features based on a parent entity, in this case the customer of a particular session. For example, the feature below calculates the mean transaction amount of the customer of the session.

### Improve feature output
To learn about the parameters to change in DFS read Tuning Deep Feature Synthesis.